In [ ]:
# 赛博炼丹有风险

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
from pylab import rcParams

In [2]:
data_cs = pd.read_csv('./data-temp/pre/changsha.csv')

```
data_cd = pd.read_csv('./data-temp/pre/chengdu.csv')
data_heb = pd.read_csv('./data-temp/pre/haerbin.csv')
data_hz = pd.read_csv('./data-temp/pre/hangzhou.csv')
data_jn = pd.read_csv('./data-temp/pre/jinan.csv')
data_wulu = pd.read_csv('./data-temp/pre/wulumuqi.csv')
data_zz = pd.read_csv('./data-temp/pre/zhengzhou.csv')
```

In [3]:
def split_date(city_data):
    new_names = ['Year','Month','Day']
    splits = city_data['Time'].str.split('-', expand=True)
    splits.columns = new_names
    city_data = city_data.join(splits)
    return city_data

In [4]:
data_cs = split_date(data_cs)

```
data_cd = split_date(data_cd)
data_heb = split_date(data_heb)
data_hz = split_date(data_hz)
data_jn = split_date(data_jn)
data_wulu = split_date(data_wulu)
data_zz = split_date(data_zz)
```

In [5]:
data_cs[['Year', 'Month', 'Day']] = data_cs[['Year', 'Month', 'Day']].astype("int64")

In [6]:
data_cs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2191 entries, 0 to 2190
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AQI     2191 non-null   float64
 1   PM2_5   2191 non-null   float64
 2   PM10    2191 non-null   float64
 3   SO2     2191 non-null   float64
 4   NO2     2191 non-null   float64
 5   CO      2191 non-null   float64
 6   O3      2191 non-null   float64
 7   Time    2191 non-null   object 
 8   Year    2191 non-null   int64  
 9   Month   2191 non-null   int64  
 10  Day     2191 non-null   int64  
dtypes: float64(7), int64(3), object(1)
memory usage: 188.4+ KB


```
data_cd[['Year', 'Month', 'Day']] = data_cd[['Year', 'Month', 'Day']].astype("int64")
data_heb[['Year', 'Month', 'Day']] = data_heb[['Year', 'Month', 'Day']].astype("int64")
data_hz[['Year', 'Month', 'Day']] = data_hz[['Year', 'Month', 'Day']].astype("int64")
data_jn[['Year', 'Month', 'Day']] = data_jn[['Year', 'Month', 'Day']].astype("int64")
data_wulu[['Year', 'Month', 'Day']] = data_wulu[['Year', 'Month', 'Day']].astype("int64")
data_zz[['Year', 'Month', 'Day']] = data_zz[['Year', 'Month', 'Day']].astype("int64")
```

In [7]:
training_set_size = 0.8
look_back = 30

In [8]:
def smooth(city_data, column_name, window_size):
    temp_data = city_data[column_name]
    temp = []
    temp_sum = 0
    for i in range(len(temp_data)):
        temp_sum += temp_data[i]
        if np.mod(i+1,window_size) == 0:
            temp.append(temp_sum/window_size)
            temp_sum = 0
    return temp

In [9]:
# MinMax_scaler
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))

# 归一化
def normalization(city_data):
    AQI_data = scaler.fit_transform(city_data.values.reshape(-1,1))
    return AQI_data

# 反归一化
def inverse_normalization(city_data):
    AQI_inverse = scaler.inverse_transform(city_data)
    return AQI_inverse

# 划分数据
def split(city_data):
    split = int(len(city_data)*training_set_size)
    train = city_data[:split]
    test = city_data[split+1:]
    return train,test

In [10]:
train_set_cs,test_set_cs = split(data_cs['AQI'])
train_set_cs = normalization(train_set_cs)
test_set_cs = normalization(test_set_cs)

In [11]:
def creat_dataset(dataset,look_back):
    data_x = []
    data_y = []
    for i in range(len(dataset)-look_back):  # 用前30个数据预测下一个数据
        data_x.append(dataset[i:i+look_back])
        data_y.append(dataset[i+look_back])
    return np.asarray(data_x).reshape(-1, 1, look_back),np.asarray(data_y).reshape(-1, 1, 1)

In [12]:
x_train_cs, y_train_cs = creat_dataset(train_set_cs,look_back)

In [13]:
epoch = 300  # 训练次数
lr = 0.01  # 学习率
hidden_layer_size = 32    # 隐藏层神经元个数
num_layers = 4  # 隐藏层层数

In [14]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN,self).__init__()
        # input_size = lookback(30)
        self.lstm = nn.LSTM(look_back,hidden_layer_size,num_layers)  # lstm层
        # output_size = 1
        self.out = nn.Linear(hidden_layer_size,1)  # 输出层
    def forward(self,x):  # 前向传播
        x1,_ = self.lstm(x)
        a,b,c = x1.shape
        # 线性层输入:二维数据，lstm输出:三维数据
        # 输出的特征维度不能变
        out = self.out(x1.view(-1,c))
        # 循环神经网络，把二维output调整成三维数据，下一次循环使用
        out1 = out.view(a,b,-1)
        return out1

In [15]:
rnn = RNN()
print(rnn)
optimizer = torch.optim.Adam(rnn.parameters(),lr = 0.02)
loss_func = nn.MSELoss()

RNN(
  (lstm): LSTM(30, 32, num_layers=4)
  (out): Linear(in_features=32, out_features=1, bias=True)
)


In [16]:
loss_list = []
def train(x_train, y_train):    
    for i in range(epoch):
        x = torch.from_numpy(x_train).type(torch.FloatTensor)
        y = torch.from_numpy(y_train).type(torch.FloatTensor)
        out = rnn(x)
        loss = loss_func(out,y)
        loss_list.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
train(x_train_cs, y_train_cs)

In [ ]:
plt.plot(k_range, inertias, 'o-')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
# 画图
rcParams['figure.figsize'] = (10,8)
def plot(title,ylabel,xlabel,data,sta=0):
    plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.grid(True)
    plt.autoscale(axis='x',tight=True)
    plt.plot(data[sta:])

def plot_AQI(city_data,window_size):
    AQI_list = smooth(city_data,'AQI',window_size)
    plot('AQI','time',AQI_list) 

In [ ]:
# 测试
rnn.eval()
# 构建数据集
x_test_cs,y_test_cs = creat_dataset(test_set_cs,look_back)

def test_rnn(x_test,y_test):
    y_pred = rnn(torch.from_numpy(x_test).type(torch.FloatTensor).to(device))

    pred = y_pred.data.cpu().numpy().reshape(-1,1)
    test = y_test.reshape(-1,1)
    pred_list = []
    test_list = []
    pred_sum = 0
    test_sum = 0

    for i in range(len(pred)):
        pred_sum += pred[i]
        test_sum += test[i]
        if np.mod(i+1,look_back)==0:
            pred_list.append(pred_sum/look_back)
            test_list.append(test_sum/look_back)
            pred_sum = 0
            test_sum = 0

test_rnn(x_test_cs,y_test_cs)

In [ ]:
time_forward = 30

# 迭代：将预测得到的第a天的数据加入数据集,用作预测第a+1天
def fut_iter(in_seq):
    x = torch.from_numpy(in_seq).type(torch.FloatTensor).to(device)
    out_data = rnn(x).to(device)
    return out_data

def pred_future(city_data,time_forward=30):
    AQI_data = normalization(city_data['AQI'])
    # 创建初始数据集
    data,_ = creat_dataset(AQI_data,look_back)
    pred = rnn(torch.from_numpy(data).type(torch.FloatTensor).to(device))
    pred = pred.data.cpu().numpy().reshape(-1,1)
    in_seq = np.delete(data[-1,0],0)
    in_seq = np.append(in_seq,pred[-1]).reshape(1,-1,look_back)
    for i in range(time_forward):
        out = fut_iter(in_seq).item()
        pred = np.append(pred,out)
        in_seq = np.delete(in_seq[-1,0],0)
        # 扩增数据集
        in_seq = np.append(in_seq,out).reshape(1,-1,look_back)
    pred = pred.reshape(-1,1)
    # 反归一化
    pred = inverse_normalization(pred)
    return pred[-time_forward:]

In [ ]:
result_data = pred_future(data_cs,time_forward)

In [ ]:
result_df = pd.DataFrame(result_data,columns=['AQI'])
result_df

In [ ]:
result_df.to_csv('re_changsha.csv',index=False)